#### Importing, Preprocessing

In [81]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import os 

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_validate, cross_val_score
from scipy import sparse
from scipy.sparse import hstack, vstack

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../scr/clean_df.csv', header=0)

In [3]:
df['date_created'] = pd.to_datetime(df['date_created'],format="%Y-%m-%d %H:%M:%S.%f%z", errors='coerce')
df['year_created'] = df['date_created'].dt.year

df['man_period'] = df['year_created'] - df['manufacture_year']
df['stk_period'] = df['year_created'] - df['stk_year']

df = df.drop(['manufacture_year', 'stk_year', 'date_created', 'date_last_seen', 'year_created'], axis=1)

In [4]:
df

,maker,model,mileage,engine_displacement,engine_power,body_type,transmission,door_count,seat_count,fuel_type,price_eur,man_period,stk_period
0,ford,galaxy,151000.0,2000.0,138.12506,compact,man,5.0,7.0,diesel,10584.75,4.0,NaN
1,skoda,octavia,143476.0,2000.0,108.62262,compact,man,5.0,5.0,diesel,8882.31,3.0,NaN
2,bmw,NaN,97676.0,1995.0,113.98670,compact,man,5.0,5.0,diesel,12065.06,5.0,NaN
3,skoda,fabia,111970.0,1200.0,84.48426,compact,man,5.0,5.0,gasoline,2960.77,11.0,NaN
4,skoda,fabia,128886.0,1200.0,84.48426,compact,man,5.0,5.0,gasoline,2738.71,11.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2834466,skoda,yeti,69267.0,1197.0,103.25854,offroad,auto,5.0,5.0,NaN,24981.50,2.0,NaN
2834467,chevrolet,matiz,59000.0,995.0,65.70998,compact,man,5.0,5.0,NaN,24981.50,10.0,NaN
2834468,chevrolet,aveo,86000.0,1200.0,71.07406,sedan,man,5.0,5.0,NaN,24981.50,6.0,NaN
2834469,ford,ka,123000.0,1299.0,68.39202,compact,man,3.0,4.0,NaN,24981.50,11.0,NaN


In [5]:
df2 = df.copy(deep=True)

In [6]:
m1 = df.memory_usage(deep=True).sum()
m1

956108316

In [7]:
# Filled NaN values with zeroes

df2.fillna(0, inplace=True)

In [8]:
numerical_columns = ['engine_power', 'mileage', 'engine_displacement']
conv_to_string_columns = ['door_count', 'seat_count', 'man_period', 'stk_period']
categorical_columns = ['maker', 'model', 'body_type', 'transmission', 'fuel_type']+conv_to_string_columns


In [9]:
def datatype_converter(x):

    for i in conv_to_string_columns:
        x[i] = x[i].apply(lambda x: str(x) if not pd.isnull(x) else x)
    
    for i in categorical_columns:
        x[i] = x[i].astype('category')
    
    x[numerical_columns] = x[numerical_columns].apply(pd.to_numeric, downcast="float")
    x['price_eur'] = x['price_eur'].apply(pd.to_numeric, downcast='float')

    return x

In [10]:
df2 = datatype_converter(df2)

In [11]:
df2.dtypes

maker                  category
model                  category
mileage                 float32
engine_displacement     float32
engine_power            float32
body_type              category
transmission           category
door_count             category
seat_count             category
fuel_type              category
price_eur               float64
man_period             category
stk_period             category
dtype: object

In [12]:
m2 = df2.memory_usage(deep=True).sum()
m2

85111439

In [106]:
X = df2.drop('price_eur', axis=1)
y = df2['price_eur']

In [110]:
Xencoded = pd.get_dummies(X, columns=categorical_columns)

In [21]:
Xencoded_cat = pd.get_dummies(X[categorical_columns])

In [19]:
X_num = X[numerical_columns]

#### Conveting df to CSR matrix

In [23]:
Xcsr_cat = sparse.csr_matrix(Xencoded_cat.values)

In [24]:
Xcsr_num = sparse.csc_matrix(X_num.values)

In [36]:
Xcsr = hstack((Xcsr_num, Xcsr_cat))

In [31]:
ycsr = sparse.csr_matrix(y.values).T

#### PCA

In [54]:
from sklearn.decomposition import TruncatedSVD

In [55]:
svd = TruncatedSVD(n_components=5).fit(Xcsr)

In [56]:
Xpca = svd.transform(Xcsr)

#### Randomized Search CV for Random Forest

SVD data set (Xpca) is used for the cells in below

In [116]:
X_train, X_test, y_train, y_test = train_test_split(Xpca, y, test_size=0.2, random_state=10)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2267576, 5), (566895, 5), (2267576,), (566895,))

In [91]:
rs_param_grid = {
    "n_estimators": list((range(10, 20))),
    "max_depth": list((range(5, 15))),
    "min_samples_split": list((range(2, 10))),
    "min_samples_leaf": list((range(1, 10))),
    "ccp_alpha": [0, 0.001, 0.01, 0.1],
    "max_samples": [0.2]
}


In [92]:
scoring_list = {'R2': metrics.make_scorer(metrics.r2_score)}

In [94]:
rf = RandomForestRegressor(random_state=10, n_jobs=-1)

rf_rs = RandomizedSearchCV(
    estimator=rf,
    param_distributions=rs_param_grid,
    scoring = 'r2',
    n_iter=10,  # Number of parameter candidate settings to sample
    verbose=2,  # The higher this is, the more messages are outputed
    random_state=10,
    refit = True
)

In [95]:
rf_rs.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ccp_alpha=0.01, max_depth=9, max_samples=0.2, min_samples_leaf=5, min_samples_split=9, n_estimators=13; total time=  18.2s
[CV] END ccp_alpha=0.01, max_depth=9, max_samples=0.2, min_samples_leaf=5, min_samples_split=9, n_estimators=13; total time=  20.8s
[CV] END ccp_alpha=0.01, max_depth=9, max_samples=0.2, min_samples_leaf=5, min_samples_split=9, n_estimators=13; total time=  23.9s
[CV] END ccp_alpha=0.01, max_depth=9, max_samples=0.2, min_samples_leaf=5, min_samples_split=9, n_estimators=13; total time=  21.6s
[CV] END ccp_alpha=0.01, max_depth=9, max_samples=0.2, min_samples_leaf=5, min_samples_split=9, n_estimators=13; total time=  22.4s
[CV] END ccp_alpha=0.001, max_depth=5, max_samples=0.2, min_samples_leaf=2, min_samples_split=3, n_estimators=13; total time=  11.6s
[CV] END ccp_alpha=0.001, max_depth=5, max_samples=0.2, min_samples_leaf=2, min_samples_split=3, n_estimators=13; total time=  12.4s
[CV] END ccp_

RandomizedSearchCV(estimator=RandomForestRegressor(n_jobs=-1, random_state=10),
                   param_distributions={'ccp_alpha': [0, 0.001, 0.01, 0.1],
                                        'max_depth': [5, 6, 7, 8, 9, 10, 11, 12,
                                                      13, 14],
                                        'max_samples': [0.2],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9],
                                        'min_samples_split': [2, 3, 4, 5, 6, 7,
                                                              8, 9],
                                        'n_estimators': [10, 11, 12, 13, 14, 15,
                                                         16, 17, 18, 19]},
                   random_state=10, scoring='r2', verbose=2)

In [96]:
rf_rs.best_score_ 

# ms=0.001 -> 
# 0.759607066616578

# ms=0.01
# 0.8019836374427675

# ms=0.1
# 0.8198670082118591

# ms=0.1 svd
# 0.7682596872865625


0.7923369462748235

In [117]:
rf_rs.best_params_

# ms=0.1
# {'n_estimators': 18,
#  'min_samples_split': 6,
#  'min_samples_leaf': 6,
#  'max_samples': 0.1,
#  'max_depth': 11,
#  'ccp_alpha': 0.1}

# ms=0.2
# {'n_estimators': 17,
#  'min_samples_split': 3,
#  'min_samples_leaf': 9,
#  'max_samples': 0.2,
#  'max_depth': 13,
#  'ccp_alpha': 0.1}

{'n_estimators': 17,
 'min_samples_split': 3,
 'min_samples_leaf': 9,
 'max_samples': 0.2,
 'max_depth': 13,
 'ccp_alpha': 0.1}

In [118]:
y_pred = rf_rs.best_estimator_.predict(X_test)
print(metrics.r2_score(y_pred, y_test)) 

# ms=0.01
# 0.6485229513084686

# ms=0.1 
# 0.7717527292153455

# ms=0.2 svd
# 0.7298531873984379

0.7298531873984379


#### Random Forest

Using best parameters from RandomizedSearchCV.  
77% R2 on test set was got from training from Xencoded data set.

In [111]:
X_train, X_test, y_train, y_test = train_test_split(Xencoded, y, test_size=0.2, random_state=10)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2267576, 921), (566895, 921), (2267576,), (566895,))

In [112]:
params = {
    'n_estimators': 18,
    'min_samples_split': 6,
    'min_samples_leaf': 6,
    'max_samples': 0.1,
    'max_depth': 11,
    'ccp_alpha': 0.1,
    "random_state": 10,
    "n_jobs": -1,
    "bootstrap": True,
}

rf = RandomForestRegressor(**params)

rf = rf.fit(X_train, y_train)

In [113]:
y_pred = rf.predict(X_test)
print(metrics.r2_score(y_pred, y_test)) #0.6485229513084686

0.7717527292153455
